<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Loading data</a></span></li><li><span><a href="#Feature-selection-/-engineering" data-toc-modified-id="Feature-selection-/-engineering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Feature selection / engineering</a></span></li><li><span><a href="#Processing-features" data-toc-modified-id="Processing-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Processing features</a></span></li><li><span><a href="#Saving-data" data-toc-modified-id="Saving-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Saving data</a></span></li></ul></div>

# Overview

This file creates the features used for LoLwinner, using data scraped using the official League of Legends API

**Table of features** (TBC)

| Feature | Description | Dimension | Values |
| --- | --- | --- |
|`winners`           |Match winner|scalar|1: Blue wins, 0: red wins
|`current_gold`      ||
|`total_gold`        ||
|`xp`                ||
|`max_current_gold`  ||
|`max_total_gold`    ||
|`max_xp`            ||
|`champions`         |Champions (characters) selected by each player|Champion_ID integer
|`kills`             |Number of opponents killed by each team|
|`buildings`         ||
|`monsters`          ||
|`num_frames`        ||
|`match_ids`         ||
|`team_stats`        ||
|`duration`          ||
|`player_tiers`      ||
|`versions`          ||
|`game_types`        ||

# Imports

In [102]:
import json
import numpy as np
np.set_printoptions(suppress=True)

import warnings
warnings.filterwarnings('ignore')

# helper functions for extraction features
from lib import feature_calculators, utils, match_factory

from importlib import reload
reload(feature_calculators);
reload(utils);
reload(match_factory);

LOG_LEVEL = 'Debug'

# Loading data

Loading the timeline data for *expert* ("challenger") level matches. This represents ~7.5K matches out of the downloaded set of ~55K.

Note that every match has two files associated with it:

1. **timeline** file:
    - Contains: Minute-level in-game data
    - Further details: [see documentation here](https://developer.riotgames.com/api-methods/#match-v3/GET_getMatchTimeline)  
<br/>
2. **match** file:
    - Contains: Metadata and other data about match (e.g., game mode, game map)
    - Further details: [see documentation here](https://developer.riotgames.com/api-methods/#match-v3/GET_getMatch)

In [96]:
# load match ids

folder = './dat_challenger_only/'

match_ids = utils.get_match_ids(folder)

utils.log(LOG_LEVEL, str(len(match_ids)) + ' files')

7539 files


In [98]:
matches = []
for i, match_id in enumerate(match_ids):
    if i % 500 == 0: utils.log(LOG_LEVEL, str(i))
    match = match_factory.load(match_id)
    if match: matches.append(match)
    
features = match_factory.transpose_matches_to_features(matches)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [ ]:
tensor_features = 

# Feature selection / engineering

Several features are selected / engineered using the **timeline** and **match** files.

The `features` hashtable stores features for each match, such that each `feature` in `features` has dimensions of

`(number of matches) x (dimension of feature)`

In [3]:
features = {
    'winners'           : [],
    'current_gold'      : [],
    'total_gold'        : [],
    'xp'                : [],
    'max_current_gold'  : [],
    'max_total_gold'    : [],
    'max_xp'            : [],
    'champions'         : [],
    'kills'             : [],
    'buildings'         : [],
    'monsters'          : [],
    'num_frames'        : [],
    'match_ids'         : [],
    'team_stats'        : [],
    'duration'          : [],
    'player_tiers'      : [],
    'versions'          : [],
    'game_types'        : []
}

# Processing features

We pad the data tensors with zeros, to simplify our analysis

In [103]:
tensor_features = feature_calculators.calculate_tensor_features(features)

**Outputting dimensions of feature tensor.**

Dimensions are of the form

`(N, M_1, M_2, ...)`

where

`N` is number of matches and

`(M_1, M_2, ...)` is dimensions of the feature

(For scalar features, dimension is `(N,)`

In [104]:
for feature in tensor_features.keys():
    print('{:<20} {:<12}'.format(feature, str(tensor_features[feature].shape)))

winners              (7302,)     
current_gold         (7302, 72, 3)
total_gold           (7302, 72, 3)
xp                   (7302, 72, 3)
max_current_gold     (7302, 72, 3)
max_total_gold       (7302, 72, 3)
max_xp               (7302, 72, 3)
champions            (7302, 10)  
kills                (7302, 72, 3)
buildings            (7302, 72, 3, 5)
monsters             (7302, 72, 3, 7)
num_frames           (7302,)     
team_stats           (7302, 6)   
duration             (7302,)     
versions             (7302,)     
game_types           (7302,)     
player_tiers         (7302, 10)  


# Saving data

In [34]:
fout = 'lolwinner_data.npz'

np.savez(fout, **tensor_features)